In [33]:
import pandas as pd
from pathlib import Path
import numpy as np
%matplotlib inline 
import hvplot.pandas

In [34]:
# Import NYC Aibrnb data
nyc_data_path = Path("../Resources/Data/AB_NYC_2019.csv")
nyc_sales_path = Path("../Resources/Data/nyc-rolling-sales.csv")

# Create the DFs 
airbnb_data = pd.read_csv(nyc_data_path, index_col="id")
display(airbnb_data.head(5))

sales_data = pd.read_csv(nyc_sales_path, infer_datetime_format=True, parse_dates=True)
display(sales_data.head(5))

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,,
2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [35]:
# Drop unwanted columns
sales_data.drop(columns={
    "Unnamed: 0", 
    "TAX CLASS AT PRESENT", 
    "BLOCK", 
    "LOT", 
    "EASE-MENT", 
    "BUILDING CLASS AT PRESENT", 
    "YEAR BUILT", 
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS AT TIME OF SALE",
    "LAND SQUARE FEET",
    "APARTMENT NUMBER", 
    "RESIDENTIAL UNITS",
    "COMMERCIAL UNITS",
    "ADDRESS",
    "BUILDING CLASS CATEGORY",
    "ZIP CODE",
    "TOTAL UNITS",
    "BOROUGH"
}, inplace=True)

sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NEIGHBORHOOD       84548 non-null  object
 1   GROSS SQUARE FEET  84548 non-null  object
 2   SALE PRICE         84548 non-null  object
 3   SALE DATE          84548 non-null  object
dtypes: object(4)
memory usage: 2.6+ MB


In [36]:
# csv shows '-' as ' -  ' in excel. Need to replace them to drop later on
sales_data.replace(' -  ', np.NaN, inplace=True)

# Drop nulls
sales_data.dropna(inplace=True)

# Drop duplicates
sales_data.drop_duplicates(inplace=True)

# Conver number values to integer types
sales_data['SALE PRICE'] = sales_data['SALE PRICE'].astype(int)
sales_data['GROSS SQUARE FEET'] = sales_data['GROSS SQUARE FEET'].astype(int)

# Drop anomalies 
sales_data = sales_data[sales_data['SALE PRICE'] > 10000]
sales_data = sales_data[sales_data['GROSS SQUARE FEET'] > 100]

# Calculate Price / SQFT
sales_data['PRICE PER SQFT'] = sales_data['SALE PRICE'] / sales_data['GROSS SQUARE FEET']

# Describe the DataFrame
display(sales_data.head(5))
sales_data.info()

,NEIGHBORHOOD,GROSS SQUARE FEET,SALE PRICE,SALE DATE,PRICE PER SQFT
0,ALPHABET CITY,6440,6625000,2017-07-19 00:00:00,1028.726708
3,ALPHABET CITY,6794,3936272,2016-09-23 00:00:00,579.374742
4,ALPHABET CITY,4615,8000000,2016-11-17 00:00:00,1733.477790
6,ALPHABET CITY,4226,3192840,2016-09-23 00:00:00,755.522953
9,ALPHABET CITY,18523,16232000,2016-11-07 00:00:00,876.315932


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28203 entries, 0 to 84547
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NEIGHBORHOOD       28203 non-null  object 
 1   GROSS SQUARE FEET  28203 non-null  int64  
 2   SALE PRICE         28203 non-null  int64  
 3   SALE DATE          28203 non-null  object 
 4   PRICE PER SQFT     28203 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.3+ MB


In [37]:
# Clean AirBnb Data
airbnb_data.columns
airbnb_data.drop(columns={
    'name',
    'host_id',
    'host_name',
    'minimum_nights',
    'number_of_reviews', 
    'last_review', 
    'reviews_per_month',
    'calculated_host_listings_count', 
    'availability_365',
    'neighbourhood_group',
    'room_type'
}, inplace=True)

airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48895 entries, 2539 to 36487245
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   neighbourhood  48895 non-null  object 
 1   latitude       48895 non-null  float64
 2   longitude      48895 non-null  float64
 3   price          48895 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 1.9+ MB


In [38]:
#Drop nulls
airbnb_data.dropna(inplace=True)

# Drop duplicates 
airbnb_data.drop_duplicates(inplace=True)

# Drop anomalies 
airbnb_data = airbnb_data[airbnb_data['price'] > 100]

# Rename columns
airbnb_data = airbnb_data.rename(columns={"price": "AirBnb price per night"})

airbnb_data.info()
display(airbnb_data.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24966 entries, 2539 to 36485431
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood           24966 non-null  object 
 1   latitude                24966 non-null  float64
 2   longitude               24966 non-null  float64
 3   AirBnb price per night  24966 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 975.2+ KB


,neighbourhood,latitude,longitude,AirBnb price per night
id,,,,
2539,Kensington,40.64749,-73.97237,149
2595,Midtown,40.75362,-73.98377,225
3647,Harlem,40.80902,-73.94190,150
5099,Murray Hill,40.74767,-73.97500,200
5238,Chinatown,40.71344,-73.99037,150


In [39]:
# Create dictionary to help normalize index names across dataframes 
# TODO: see if there's a way to read in a text file as a dictionary
dictionary = {
'AIRPORT LA GUARDIA':np.NaN,
'Allerton':np.NaN,
'ALPHABET CITY':np.NaN,
'ANNADALE':np.NaN,
'Arden Heights':'Arden Heights',
'ARDEN HEIGHTS':'Arden Heights',
'Arrochar':'Arrochar',
'ARROCHAR':'Arrochar',
'ARROCHAR-SHORE ACRES':'Arrochar',
'Arverne':'Arverne',
'ARVERNE':'Arverne',
'Astoria':'Astoria',
'ASTORIA':'Astoria',
'Bath Beach':'Bath Beach',
'BATH BEACH':'Bath Beach',
'BATHGATE':np.NaN,
'Battery Park City':np.NaN,
'Bay Ridge':'Bay Ridge',
'BAY RIDGE':'Bay Ridge',
'Bay Terrace':'Bay Terrace',
'Bay Terrace, Staten Island':'Bay Terrace',
'Baychester':'Baychester',
'BAYCHESTER':'Baychester',
'Bayside':'Bayside',
'BAYSIDE':'Bayside',
'Bayswater':np.NaN,
'BEDFORD PARK/NORWOOD':'Bedford-Stuyvesant',
'BEDFORD STUYVESANT':'Bedford-Stuyvesant',
'Bedford-Stuyvesant':'Bedford-Stuyvesant',
'BEECHHURST':np.NaN,
'Belle Harbor':'Belle Harbor',
'BELLE HARBOR':'Belle Harbor',
'Bellerose':'Bellerose',
'BELLEROSE':'Bellerose',
'Belmont':'Belmont',
'BELMONT':'Belmont',
'Bensonhurst':'Bensonhurst',
'BENSONHURST':'Bensonhurst',
'Bergen Beach':'Bergen Beach',
'BERGEN BEACH':'Bergen Beach',
'BLOOMFIELD':np.NaN,
'Boerum Hill':'Boerum Hill',
'BOERUM HILL':'Boerum Hill',
'Borough Park':'Borough Park',
'BOROUGH PARK':'Borough Park',
'Breezy Point':np.NaN,
'Briarwood':'Briarwood',
'BRIARWOOD':'Briarwood',
'Brighton Beach':'Brighton Beach',
'BRIGHTON BEACH':'Brighton Beach',
'BROAD CHANNEL':'BROAD CHANNEL',
'BRONX PARK':'BROAD CHANNEL',
'Bronxdale':'Bronxdale',
'BRONXDALE':'BRONXDALE',
'Brooklyn Heights':'Brooklyn Heights',
'BROOKLYN HEIGHTS':'BROOKLYN HEIGHTS',
'Brownsville':'Brownsville',
'BROWNSVILLE':'Brownsville',
"Bull's Head":"Bull's Head",
"BULLS HEAD":"Bull's Head",
'BUSH TERMINAL':np.NaN,
'Bushwick':'Bushwick',
'BUSHWICK':'Bushwick',
'Cambria Heights':'Cambria Heights',
'CAMBRIA HEIGHTS':'Cambria Heights',
'Canarsie':'Canarsie',
'CANARSIE':'Canarsie',
'Carroll Gardens':'Carroll Gardens',
'CARROLL GARDENS':'Carroll Gardens',
'Castle Hill':'Castle Hill',
'CASTLE HILL/UNIONPORT':'Castle Hill',
'Castleton Corners':'Castleton Corners',
'CASTLETON CORNERS':'Castleton Corners',
'Chelsea':'Chelsea',
'CHELSEA':'Chelsea',
'Chinatown':'Chinatown',
'CHINATOWN':'Chinatown',
'City Island':'City Island',
'CITY ISLAND':'City Island',
'CITY ISLAND-PELHAM STRIP':'City Island',
'Civic Center':'Civic Center',
'CIVIC CENTER':'Civic Center',
'Claremont Village':np.NaN,
'Clason Point':np.NaN,
'Clifton':np.NaN,
'CLINTON':'CLINTON',
'Clinton Hill':'CLINTON',
'CLINTON HILL':'CLINTON',
'CLOVE LAKES':np.NaN,
'Co-op City':'Co-op City',
'CO-OP CITY':'Co-op City',
'Cobble Hill':'Cobble Hill',
'COBBLE HILL':'Cobble Hill',
'COBBLE HILL-WEST':'Cobble Hill',
'College Point':'College Point',
'COLLEGE POINT':'College Point',
'Columbia St':np.NaN,
'Concord':'Concord',
'CONCORD':'Concord',
'CONCORD-FOX HILLS':'Concord',
'Concourse':'Concourse',
'Concourse Village':'Concourse',
'Coney Island':'Coney Island',
'CONEY ISLAND':'Coney Island',
'Corona':'Corona',
'CORONA':'Corona',
'COUNTRY CLUB':np.NaN,
'CROTONA PARK':np.NaN,
'Crown Heights':'Crown Heights',
'CROWN HEIGHTS':'Crown Heights',
'Cypress Hills':'Cypress Hills',
'CYPRESS HILLS':'Cypress Hills',
'Ditmars Steinway':'Ditmars Steinway',
'Dongan Hills':'Ditmars Steinway',
'DONGAN HILLS':'DONGAN HILLS',
'DONGAN HILLS-COLONY':'DONGAN HILLS',
'DONGAN HILLS-OLD TOWN':'DONGAN HILLS',
'Douglaston':'Douglaston',
'DOUGLASTON':'Douglaston',
'Downtown Brooklyn':'Downtown Brooklyn',
'DOWNTOWN-FULTON FERRY':'Downtown Brooklyn',
'DOWNTOWN-FULTON MALL':'Downtown Brooklyn',
'DOWNTOWN-METROTECH':'Downtown Brooklyn',
'DUMBO':np.NaN,
'Dyker Heights':'Dyker Heights',
'DYKER HEIGHTS':'Dyker Heights',
'East Elmhurst':'East Elmhurst',
'EAST ELMHURST':'East Elmhurst',
'East Flatbush':'Flatbush',
'East Harlem':'Harlem',
'East Morrisania':np.NaN,
'East New York':'East New York',
'EAST NEW YORK':'East New York',
'EAST RIVER':'East New York',
'EAST TREMONT':np.NaN,
'East Village':'East Village',
'EAST VILLAGE':'East Village',
'Eastchester':np.NaN,
'Edenwald':np.NaN,
'Edgemere':np.NaN,
'Elmhurst':'Elmhurst',
'ELMHURST':'Elmhurst',
'Eltingville':'Eltingville',
'ELTINGVILLE':'Eltingville',
'Emerson Hill':'Emerson Hill',
'EMERSON HILL':'Emerson Hill',
'Far Rockaway':'Far Rockaway',
'FAR ROCKAWAY':'Far Rockaway',
'FASHION':np.NaN,
'Fieldston':'Fieldston',
'FIELDSTON':'Fieldston',
'FINANCIAL':'Financial District',
'Financial District':'Financial District',
'Flatbush':'Flatbush',
'FLATBUSH-CENTRAL':'Flatbush',
'FLATBUSH-EAST':'Flatbush',
'FLATBUSH-LEFFERTS GARDEN':'Flatbush',
'FLATBUSH-NORTH':'Flatbush',
'FLATIRON':'Flatiron District',
'Flatiron District':'Flatiron District',
'Flatlands':'Flatlands',
'FLATLANDS':'Flatlands',
'FLORAL PARK':np.NaN,
'Flushing':'Flushing',
'FLUSHING MEADOW PARK':'Flushing',
'FLUSHING-NORTH':'Flushing',
'FLUSHING-SOUTH':'Flushing',
'Fordham':'Fordham',
'FORDHAM':'Fordham',
'Forest Hills':'Forest Hills',
'FOREST HILLS':'Forest Hills',
'Fort Greene':'Fort Greene',
'FORT GREENE':'Fort Greene',
'Fort Hamilton':np.NaN,
'Fort Wadsworth':np.NaN,
'FRESH KILLS':np.NaN,
'Fresh Meadows':'Fresh Meadows',
'FRESH MEADOWS':'Fresh Meadows',
'GERRITSEN BEACH':np.NaN,
'GLEN OAKS':np.NaN,
'Glendale':'Glendale',
'GLENDALE':'Glendale',
'Gowanus':'Gowanus',
'GOWANUS':'Gowanus',
'Gramercy':'Gramercy',
'GRAMERCY':'Gramercy',
'Graniteville':np.NaN,
'Grant City':'Grant City',
'GRANT CITY':'Grant City',
'GRASMERE':np.NaN,
'Gravesend':'Gravesend',
'GRAVESEND':'Gravesend',
'Great Kills':'Great Kills',
'GREAT KILLS':'Great Kills',
'GREAT KILLS-BAY TERRACE':'Great Kills',
'Greenpoint':'Greenpoint',
'GREENPOINT':'Greenpoint',
'Greenwich Village':'Greenwich Village',
'GREENWICH VILLAGE-CENTRAL':'Greenwich Village',
'GREENWICH VILLAGE-WEST':'Greenwich Village',
'Grymes Hill':'Grymes Hill',
'GRYMES HILL':'Grymes Hill',
'HAMMELS':np.NaN,
'Harlem':'Harlem',
'HARLEM-CENTRAL':'Harlem',
'HARLEM-EAST':'Harlem',
'HARLEM-UPPER':'Harlem',
'HARLEM-WEST':'Harlem',
"Hell's Kitchen":np.NaN,
'Highbridge':'Highbridge',
'HIGHBRIDGE/MORRIS HEIGHTS':'Highbridge',
'HILLCREST':np.NaN,
'Hollis':'Hollis',
'HOLLIS':'Hollis',
'HOLLIS HILLS':'Hollis',
'Holliswood':'Holliswood',
'HOLLISWOOD':'Holliswood',
'Howard Beach':'Howard Beach',
'HOWARD BEACH':'HOWARD BEACH',
'Howland Hook':np.NaN,
'Huguenot':np.NaN,
'HUGUENOT':np.NaN,
'Hunts Point':'Hunts Point',
'HUNTS POINT':'Hunts Point',
'Inwood':'Inwood',
'INWOOD':'Inwood',
'Jackson Heights':'Jackson Heights',
'JACKSON HEIGHTS':'Jackson Heights',
'Jamaica':'Jamaica',
'JAMAICA':'Jamaica',
'JAMAICA BAY':'Jamaica',
'Jamaica Estates':'Jamaica',
'JAMAICA ESTATES':'Jamaica',
'Jamaica Hills':'Jamaica',
'JAMAICA HILLS':'Jamaica',
'JAVITS CENTER':np.NaN,
'Kensington':'Kensington',
'KENSINGTON':'Kensington',
'Kew Gardens':'Kew Gardens',
'KEW GARDENS':'Kew Gardens',
'Kew Gardens Hills':'Kew Gardens',
'Kingsbridge':'Kingsbridge',
'KINGSBRIDGE HTS/UNIV HTS':'Kingsbridge',
'KINGSBRIDGE/JEROME PARK':'Kingsbridge',
'Kips Bay':'Kips Bay',
'KIPS BAY':'Kips Bay',
'Laurelton':'Laurelton',
'LAURELTON':'Laurelton',
'Lighthouse Hill':np.NaN,
'Little Italy':'Little Italy',
'LITTLE ITALY':'Little Italy',
'Little Neck':'Little Neck',
'LITTLE NECK':'Little Neck',
'LIVINGSTON':np.NaN,
'Long Island City':'Long Island City',
'LONG ISLAND CITY':'Long Island City',
'Longwood':np.NaN,
'Lower East Side':'Lower East Side',
'LOWER EAST SIDE':'Lower East Side',
'MADISON':np.NaN,
'Manhattan Beach':'Manhattan Beach',
'MANHATTAN BEACH':'Manhattan Beach',
'MANHATTAN VALLEY':np.NaN,
'MANOR HEIGHTS':np.NaN,
'Marble Hill':np.NaN,
'MARINE PARK':np.NaN,
'Mariners Harbor':'Mariners Harbor',
'MARINERS HARBOR':'Mariners Harbor',
'Maspeth':'Maspeth',
'MASPETH':'Maspeth',
'Melrose':'Melrose',
'MELROSE/CONCOURSE':'Melrose',
'Middle Village':'Middle Village',
'MIDDLE VILLAGE':'Middle Village',
'Midland Beach':'Midland Beach',
'MIDLAND BEACH':'Midland Beach',
'Midtown':'Midtown',
'MIDTOWN CBD':'Midtown',
'MIDTOWN EAST':'Midtown',
'MIDTOWN WEST':'Midtown',
'Midwood':'Midwood',
'MIDWOOD':'Midwood',
'Mill Basin':'Mill Basin',
'MILL BASIN':'Mill Basin',
'Morningside Heights':'Morningside Heights',
'MORNINGSIDE HEIGHTS':'Morningside Heights',
'Morris Heights':'Morris Heights',
'Morris Park':'Morris Heights',
'MORRIS PARK/VAN NEST':'Morris Heights',
'Morrisania':'Morrisania',
'MORRISANIA/LONGWOOD':'Morrisania',
'Mott Haven':'Mott Haven',
'MOTT HAVEN/PORT MORRIS':'Mott Haven',
'Mount Eden':'Mount Eden',
'Mount Hope':'Mount Eden',
'MOUNT HOPE/MOUNT EDEN':'Mount Eden',
'Murray Hill':'Murray Hill',
'MURRAY HILL':'Murray Hill',
'Navy Yard':'Navy Yard',
'NAVY YARD':'Navy Yard',
'Neponsit':'Neponsit',
'NEPONSIT':'Neponsit',
'New Brighton':'New Brighton',
'NEW BRIGHTON':'New Brighton',
'NEW BRIGHTON-ST. GEORGE':'New Brighton',
'New Dorp':'New Dorp',
'NEW DORP':'New Dorp',
'New Dorp Beach':'New Dorp',
'NEW DORP-BEACH':'New Dorp',
'NEW DORP-HEIGHTS':'New Dorp',
'New Springville':'New Springville',
'NEW SPRINGVILLE':'New Springville',
'NoHo':np.NaN,
'Nolita':np.NaN,
'North Riverdale':np.NaN,
'Norwood':np.NaN,
'OAKLAND GARDENS':np.NaN,
'Oakwood':'Oakwood',
'OAKWOOD':'Oakwood',
'OAKWOOD-BEACH':'Oakwood',
'OCEAN HILL':np.NaN,
'OCEAN PARKWAY-NORTH':np.NaN,
'OCEAN PARKWAY-SOUTH':np.NaN,
'OLD MILL BASIN':np.NaN,
'Olinville':np.NaN,
'Ozone Park':'Ozone Park',
'OZONE PARK':'Ozone Park',
'Park Slope':'Park Slope',
'PARK SLOPE':'Park Slope',
'PARK SLOPE SOUTH':'Park Slope',
'Parkchester':'Parkchester',
'PARKCHESTER':'Parkchester',
'Pelham Bay':'Pelham Bay',
'PELHAM BAY':'Pelham Bay',
'Pelham Gardens':'Pelham Gardens',
'PELHAM GARDENS':'Pelham Gardens',
'PELHAM PARKWAY NORTH':'Pelham Gardens',
'PELHAM PARKWAY SOUTH':'Pelham Gardens',
'PLEASANT PLAINS':np.NaN,
'PORT IVORY':np.NaN,
'Port Morris':np.NaN,
'Port Richmond':'Port Richmond',
'PORT RICHMOND':'Port Richmond',
"Prince's Bay":"Prince's Bay",
'PRINCES BAY':"Prince's Bay",
'Prospect Heights':'Prospect Heights',
'PROSPECT HEIGHTS':'Prospect Heights',
'Prospect-Lefferts Gardens':'Prospect Heights',
'Queens Village':'Queens Village',
'QUEENS VILLAGE':'Queens Village',
'Randall Manor':np.NaN,
'Red Hook':'Red Hook',
'RED HOOK':'Red Hook',
'Rego Park':'Rego Park',
'REGO PARK':'Rego Park',
'Richmond Hill':'Richmond Hill',
'RICHMOND HILL':'Richmond Hill',
'Richmondtown':'Richmondtown',
'RICHMONDTOWN':'Richmondtown',
'RICHMONDTOWN-LIGHTHS HILL':'Richmondtown',
'Ridgewood':'Ridgewood',
'RIDGEWOOD':'Ridgewood',
'Riverdale':'Riverdale',
'RIVERDALE':'Riverdale',
'Rockaway Beach':'Rockaway Beach',
'ROCKAWAY PARK':'Rockaway Beach',
'Roosevelt Island':'Roosevelt Island',
'ROOSEVELT ISLAND':'Roosevelt Island',
'Rosebank':'Rosebank',
'ROSEBANK':'Rosebank',
'Rosedale':'Rosedale',
'ROSEDALE':'Rosedale',
'Rossville':'Rossville',
'ROSSVILLE':'Rossville',
'ROSSVILLE-CHARLESTON':'Rossville',
'ROSSVILLE-PORT MOBIL':'Rossville',
'ROSSVILLE-RICHMOND VALLEY':'Rossville',
'Schuylerville':'Schuylerville',
'SCHUYLERVILLE/PELHAM BAY':'Schuylerville',
'Sea Gate':'Sea Gate',
'SEAGATE':'Sea Gate',
'Sheepshead Bay':'Sheepshead Bay',
'SHEEPSHEAD BAY':'Sheepshead Bay',
'Shore Acres':np.NaN,
'Silver Lake':'Silver Lake',
'SILVER LAKE':'Silver Lake',
'SO. JAMAICA-BAISLEY PARK':np.NaN,
'SoHo':'SoHo',
'SOHO':'SoHo',
'Soundview':'Soundview',
'SOUNDVIEW':'Soundview',
'South Beach':'South Beach',
'SOUTH BEACH':'South Beach',
'SOUTH JAMAICA':np.NaN,
'South Ozone Park':'South Ozone Park',
'SOUTH OZONE PARK':'South Ozone Park',
'South Slope':np.NaN,
'SOUTHBRIDGE':np.NaN,
'SPRING CREEK':np.NaN,
'Springfield Gardens':'Springfield Gardens',
'SPRINGFIELD GARDENS':'Springfield Gardens',
'Spuyten Duyvil':np.NaN,
'St. Albans':'St. Albans',
'ST. ALBANS':'St. Albans',
'St. George':np.NaN,
'Stapleton':'Stapleton',
'STAPLETON':'Stapleton',
'STAPLETON-CLIFTON':'Stapleton',
'Stuyvesant Town':'Bedford-Stuyvesant',
'Sunnyside':'Sunnyside',
'SUNNYSIDE':'Sunnyside',
'Sunset Park':'Sunset Park',
'SUNSET PARK':'Sunset Park',
'Theater District':np.NaN,
'Throgs Neck':'Throgs Neck',
'THROGS NECK':'Throgs Neck',
'Todt Hill':'Todt Hill',
'TODT HILL':'Todt Hill',
'Tompkinsville':'Tompkinsville',
'TOMPKINSVILLE':'Tompkinsville',
'Tottenville':'Tottenville',
'TOTTENVILLE':'Tottenville',
'TRAVIS':np.NaN,
'Tremont':np.NaN,
'Tribeca':'Tribeca',
'TRIBECA':'Tribeca',
'Two Bridges':np.NaN,
'Unionport':np.NaN,
'University Heights':np.NaN,
'Upper East Side':'Upper East Side',
'UPPER EAST SIDE (59-79)':'Upper East Side',
'UPPER EAST SIDE (79-96)':'Upper East Side',
'UPPER EAST SIDE (96-110)':'Upper East Side',
'Upper West Side':'Upper West Side',
'UPPER WEST SIDE (59-79)':'Upper West Side',
'UPPER WEST SIDE (79-96)':'Upper West Side',
'UPPER WEST SIDE (96-116)':'Upper West Side',
'VAN CORTLANDT PARK':np.NaN,
'Van Nest':np.NaN,
'Vinegar Hill':np.NaN,
'Wakefield':'Wakefield',
'WAKEFIELD':'Wakefield',
'Washington Heights':'Washington Heights',
'WASHINGTON HEIGHTS LOWER':'Washington Heights',
'WASHINGTON HEIGHTS UPPER':'Washington Heights',
'West Brighton':np.NaN,
'West Farms':np.NaN,
'WEST NEW BRIGHTON':np.NaN,
'West Village':np.NaN,
'WESTCHESTER':'Westchester Square',
'Westchester Square':'Westchester Square',
'Westerleigh':'Westerleigh',
'WESTERLEIGH':'Westerleigh',
'Whitestone':'Whitestone',
'WHITESTONE':'Whitestone',
'Williamsbridge':'Williamsbridge',
'WILLIAMSBRIDGE':'Williamsbridge',
'Williamsburg':'Williamsburg',
'WILLIAMSBURG-CENTRAL':'Williamsburg',
'WILLIAMSBURG-EAST':'Williamsburg',
'WILLIAMSBURG-NORTH':'Williamsburg',
'WILLIAMSBURG-SOUTH':'Williamsburg',
'Willowbrook':'Willowbrook',
'WILLOWBROOK':'Willowbrook',
'Windsor Terrace':'Windsor Terrace',
'WINDSOR TERRACE':'Windsor Terrace',
'Woodhaven':'Woodhaven',
'WOODHAVEN':'Woodhaven',
'Woodlawn':'Woodlawn',
'WOODLAWN':'Woodlawn',
'Woodrow':'Woodrow',
'WOODROW':'Woodrow',
'Woodside':'Woodside',
'WOODSIDE':'Woodside',
'WYCKOFF HEIGHTS':np.NaN
}

In [40]:
# Normalize neighborhood names across DataFrames 
sales_data['NEIGHBORHOOD'] = sales_data['NEIGHBORHOOD'].replace(dictionary)

sales_data.dropna(inplace=True)

display(sales_data.head(5))
display(sales_data.tail(5))

,NEIGHBORHOOD,GROSS SQUARE FEET,SALE PRICE,SALE DATE,PRICE PER SQFT
206,Chelsea,3855,7425000,2016-12-30 00:00:00,1926.070039
212,Chelsea,4978,7750000,2017-04-17 00:00:00,1556.850141
214,Chelsea,3725,8300000,2017-02-02 00:00:00,2228.187919
216,Chelsea,3762,16000000,2016-09-08 00:00:00,4253.056885
218,Chelsea,2890,5950000,2016-10-26 00:00:00,2058.823529


,NEIGHBORHOOD,GROSS SQUARE FEET,SALE PRICE,SALE DATE,PRICE PER SQFT
84543,Woodrow,2575,450000,2016-11-28 00:00:00,174.757282
84544,Woodrow,2377,550000,2017-04-21 00:00:00,231.384098
84545,Woodrow,1496,460000,2017-07-05 00:00:00,307.486631
84546,Woodrow,64117,11693337,2016-12-21 00:00:00,182.374986
84547,Woodrow,2400,69300,2016-10-27 00:00:00,28.875000


In [41]:
# Normalize neighborhood names across DataFrames 
airbnb_data['neighbourhood'] = airbnb_data['neighbourhood'].replace(dictionary)

airbnb_data.dropna(inplace=True)

display(airbnb_data.head(5))
display(airbnb_data.tail(5))

,neighbourhood,latitude,longitude,AirBnb price per night
id,,,,
2539,Kensington,40.64749,-73.97237,149
2595,Midtown,40.75362,-73.98377,225
3647,Harlem,40.80902,-73.94190,150
5099,Murray Hill,40.74767,-73.97500,200
5238,Chinatown,40.71344,-73.99037,150


,neighbourhood,latitude,longitude,AirBnb price per night
id,,,,
36480292,Williamsburg,40.71728,-73.94394,120
36481315,Williamsburg,40.71232,-73.94220,120
36482783,Williamsburg,40.71790,-73.96238,190
36483010,Midtown,40.75561,-73.96723,200
36485431,Harlem,40.81475,-73.94867,115


In [42]:
# Display Neighborhood names for each DF 
sales_data.sort_values(by="NEIGHBORHOOD", ascending=True, inplace=True)
airbnb_data.sort_values(by='neighbourhood', ascending=True, inplace=True)

display(sales_data.head(5))
display(airbnb_data.head(5))

# conver to upper case for consistency 
sales_data['NEIGHBORHOOD'] = sales_data['NEIGHBORHOOD'].str.upper()
airbnb_data['neighbourhood'] = airbnb_data['neighbourhood'].str.upper() 

display(airbnb_data.head(5))
display(sales_data.head(5))

,NEIGHBORHOOD,GROSS SQUARE FEET,SALE PRICE,SALE DATE,PRICE PER SQFT
76523,Arden Heights,1152,295000,2017-07-11 00:00:00,256.076389
76495,Arden Heights,1176,465000,2016-12-30 00:00:00,395.408163
76496,Arden Heights,1900,540000,2017-06-21 00:00:00,284.210526
76497,Arden Heights,2545,499999,2016-09-14 00:00:00,196.463261
76498,Arden Heights,2232,536000,2016-12-20 00:00:00,240.143369


,neighbourhood,latitude,longitude,AirBnb price per night
id,,,,
17716523,Arrochar,40.59347,-74.06914,122
35164652,Arrochar,40.59125,-74.08047,195
259946,Arrochar,40.59262,-74.06659,125
258838,Arrochar,40.59251,-74.06479,250
738588,Arrochar,40.59193,-74.06476,625


,neighbourhood,latitude,longitude,AirBnb price per night
id,,,,
17716523,ARROCHAR,40.59347,-74.06914,122
35164652,ARROCHAR,40.59125,-74.08047,195
259946,ARROCHAR,40.59262,-74.06659,125
258838,ARROCHAR,40.59251,-74.06479,250
738588,ARROCHAR,40.59193,-74.06476,625


,NEIGHBORHOOD,GROSS SQUARE FEET,SALE PRICE,SALE DATE,PRICE PER SQFT
76523,ARDEN HEIGHTS,1152,295000,2017-07-11 00:00:00,256.076389
76495,ARDEN HEIGHTS,1176,465000,2016-12-30 00:00:00,395.408163
76496,ARDEN HEIGHTS,1900,540000,2017-06-21 00:00:00,284.210526
76497,ARDEN HEIGHTS,2545,499999,2016-09-14 00:00:00,196.463261
76498,ARDEN HEIGHTS,2232,536000,2016-12-20 00:00:00,240.143369


In [43]:
# Group dataframes by Neighborhood
sales_data_neighborhood = sales_data.groupby(['NEIGHBORHOOD', 'SALE DATE']).mean()
sales_data_neighborhood.reset_index(inplace=True)
sales_data_neighborhood.set_index('NEIGHBORHOOD', inplace=True)

airbnb_data_neighborhood = airbnb_data.groupby(['neighbourhood']).mean()
display(sales_data_neighborhood.head())
display(airbnb_data_neighborhood.head())

,SALE DATE,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT
NEIGHBORHOOD,,,,
ARDEN HEIGHTS,2016-09-02 00:00:00,1510.0,346000.0,229.139073
ARDEN HEIGHTS,2016-09-07 00:00:00,1181.0,310000.0,272.681912
ARDEN HEIGHTS,2016-09-08 00:00:00,1882.5,445930.0,242.168047
ARDEN HEIGHTS,2016-09-13 00:00:00,1209.0,334500.0,281.266312
ARDEN HEIGHTS,2016-09-14 00:00:00,2545.0,499999.0,196.463261


,latitude,longitude,AirBnb price per night
neighbourhood,,,
ARROCHAR,40.593611,-74.068967,221.375000
ARVERNE,40.591684,-73.794277,260.780488
ASTORIA,40.764271,-73.920833,212.976974
BATH BEACH,40.601757,-74.006200,149.666667
BAY RIDGE,40.631853,-74.025842,313.840909


In [44]:
# Combine sales and airbnb cleaned data
combined_data = pd.merge(
    sales_data_neighborhood, 
    airbnb_data_neighborhood, 
    right_on='neighbourhood',
    left_index=True,
    how="inner"
)

# combined_data = pd.concat([sales_data_neighborhood, airbnb_data_neighborhood], axis="columns", join="inner")

display(combined_data.head(5))
display(combined_data.tail(5))


,SALE DATE,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night
neighbourhood,,,,,,,
ARROCHAR,2016-09-02 00:00:00,1433.0,357050.0,270.755746,40.593611,-74.068967,221.375
ARROCHAR,2016-09-08 00:00:00,2246.0,715000.0,318.343722,40.593611,-74.068967,221.375
ARROCHAR,2016-09-26 00:00:00,1104.0,350000.0,317.028986,40.593611,-74.068967,221.375
ARROCHAR,2016-10-14 00:00:00,1760.0,645000.0,366.477273,40.593611,-74.068967,221.375
ARROCHAR,2016-10-24 00:00:00,1106.0,570000.0,515.370705,40.593611,-74.068967,221.375


,SALE DATE,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night
neighbourhood,,,,,,,
WOODSIDE,2017-07-19 00:00:00,2016.0,1100000.0,545.634921,40.746251,-73.902889,179.074074
WOODSIDE,2017-07-21 00:00:00,3265.0,1200000.0,367.534456,40.746251,-73.902889,179.074074
WOODSIDE,2017-07-27 00:00:00,1548.0,842500.0,544.250646,40.746251,-73.902889,179.074074
WOODSIDE,2017-07-28 00:00:00,1320.0,915000.0,693.181818,40.746251,-73.902889,179.074074
WOODSIDE,2017-08-29 00:00:00,1584.0,750000.0,473.484848,40.746251,-73.902889,179.074074


In [45]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14265 entries, ARROCHAR to WOODSIDE
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SALE DATE               14265 non-null  object 
 1   GROSS SQUARE FEET       14265 non-null  float64
 2   SALE PRICE              14265 non-null  float64
 3   PRICE PER SQFT          14265 non-null  float64
 4   latitude                14265 non-null  float64
 5   longitude               14265 non-null  float64
 6   AirBnb price per night  14265 non-null  float64
dtypes: float64(6), object(1)
memory usage: 891.6+ KB


## Analyze basic data 
1. Plot sale price per neighborhood (geoviews) - done
2. Plot airbnb rental price by neighborhood (geoviews) - done
3. Price per airbnb - by neighborhood (per night) - X

In [46]:
# Group dataframes by Neighborhood
sales_data_neighborhood_nodate = sales_data.groupby(['NEIGHBORHOOD']).mean()
sales_data_neighborhood_nodate.reset_index(inplace=True)
sales_data_neighborhood_nodate.set_index('NEIGHBORHOOD', inplace=True)

# combine dataframes without sale date
combined_data_nodate = pd.merge(
    sales_data_neighborhood_nodate, 
    airbnb_data_neighborhood, 
    right_on='neighbourhood',
    left_index=True,
    how="inner"
)

# combined_data = pd.concat([sales_data_neighborhood, airbnb_data_neighborhood], axis="columns", join="inner")

display(combined_data_nodate.head(5))
display(combined_data_nodate.tail(5))


/var/folders/82/szvr1kmj0hl1j121k98bfvch0000gn/T/ipykernel_7513/3243627300.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sales_data_neighborhood_nodate = sales_data.groupby(['NEIGHBORHOOD']).mean()


,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night
neighbourhood,,,,,,
ARROCHAR,1895.289474,5.241142e+05,316.796721,40.593611,-74.068967,221.375000
ARVERNE,2035.346154,4.067329e+05,203.628364,40.591684,-73.794277,260.780488
ASTORIA,3890.372990,1.955089e+06,557.345238,40.764271,-73.920833,212.976974
BATH BEACH,2522.840000,1.143501e+06,501.792135,40.601757,-74.006200,149.666667
BAY RIDGE,2783.071970,1.329507e+06,532.461317,40.631853,-74.025842,313.840909


,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night
neighbourhood,,,,,,
WILLOWBROOK,1779.271930,6.283109e+05,369.205874,40.598860,-74.132170,249.000000
WINDSOR TERRACE,2058.428571,1.561744e+06,839.178149,40.654721,-73.977925,188.422222
WOODHAVEN,1939.664286,6.072024e+05,329.183791,40.693111,-73.856124,163.000000
WOODROW,2651.788889,6.855613e+05,295.153651,40.538840,-74.198260,700.000000
WOODSIDE,4021.528736,1.638936e+06,440.056212,40.746251,-73.902889,179.074074


In [47]:
# Calulate and plot sale price per neighborhood using Geoviews
sale_price_per_neighborhood_plot = combined_data_nodate.hvplot.points(
    "longitude", 
    "latitude", 
    geo=True,
    size='PRICE PER SQFT',
    color='PRICE PER SQFT',
    tiles="ESRI", 
    scale=0.3,
    frame_width=650, 
    frame_height=450,
    title='Housing Sale Price per Neighborhood (2017-2019)'
)
sale_price_per_neighborhood_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (PRICE PER SQFT)

In [48]:
# Calulate and plot airbnb revenue (per night) by neighborhood using Geoviews
airbnb_revenue_per_neighborhood_plot = combined_data_nodate.hvplot.points(
    "longitude", 
    "latitude", 
    geo=True,
    size='AirBnb price per night',
    color='AirBnb price per night',
    tiles="ESRI", 
    scale=0.6,
    frame_width=650, 
    frame_height=450,
    title='Airbnb Price per Night by Neighborhood'
)
airbnb_revenue_per_neighborhood_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (AirBnb price per night)

In [49]:
sale_airbnb_per_neighborhood_plot = combined_data_nodate.hvplot.points(
    "longitude", 
    "latitude", 
    geo=True,
    size='AirBnb price per night',
    color='PRICE PER SQFT',
    tiles="ESRI", 
    scale=0.6,
    frame_width=650, 
    frame_height=450,
    title='Sale Price vs. Airbnb Revenue (per night) by Neighborhood'
)
sale_airbnb_per_neighborhood_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (PRICE PER SQFT,AirBnb price per night)

In [50]:
airbnb_sale_sqft_per_neighborhood_year_plot = combined_data_nodate.hvplot.scatter(
    x="PRICE PER SQFT",
    xlabel="Average sale price per square foot",
    y='AirBnb price per night',
    ylabel="Airbnb average price per night",
    hover_cols='all',
    by='neighbourhood',
    height=1000,
    width=900,
    xformatter='%.0f',
    title = 'Sale price per SQFT vs. Airbnb Revenue per Night (Avg) by Neighborhood',
    fontscale = .8,
).opts(
legend_cols=7,
legend_position='bottom',
fontsize={'legend': 6,'title': 16}
)
airbnb_sale_sqft_per_neighborhood_year_plot

:NdOverlay   [neighbourhood]
   :Scatter   [PRICE PER SQFT]   (AirBnb price per night,neighbourhood,GROSS SQUARE FEET,SALE PRICE,latitude,longitude)

## Analyze Top 25 options
4. Return on airbnb - revenue over a year (overlay rental price bar, line chart cost per sqft by date)
5. Determine sale price trend over neighborhood (line chart over the years)
6. Sales price per year over rental revenue per year graph 

In [51]:
# Create dataframe of top 25 options
combined_data_nodate['Average sale vs. Airbnb price Ratio'] = combined_data_nodate['AirBnb price per night']/combined_data_nodate['PRICE PER SQFT']
display(combined_data_nodate.head(5))

,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night,Average sale vs. Airbnb price Ratio
neighbourhood,,,,,,,
ARROCHAR,1895.289474,5.241142e+05,316.796721,40.593611,-74.068967,221.375000,0.698792
ARVERNE,2035.346154,4.067329e+05,203.628364,40.591684,-73.794277,260.780488,1.280669
ASTORIA,3890.372990,1.955089e+06,557.345238,40.764271,-73.920833,212.976974,0.382128
BATH BEACH,2522.840000,1.143501e+06,501.792135,40.601757,-74.006200,149.666667,0.298264
BAY RIDGE,2783.071970,1.329507e+06,532.461317,40.631853,-74.025842,313.840909,0.589415


In [52]:
# Isolate Top 25 neighborhood options
combined_data_nodate.sort_values(by='Average sale vs. Airbnb price Ratio', ascending=False, inplace=True)

top_25_neighborhoods = combined_data_nodate[0:24]
top_25_neighborhoods

,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night,Average sale vs. Airbnb price Ratio
neighbourhood,,,,,,,
WESTCHESTER SQUARE,3146.084337,7.039822e+05,232.697507,40.843780,-73.844690,670.000000,2.879275
SEA GATE,2722.170732,7.551646e+05,285.298160,40.577053,-74.008880,787.000000,2.758518
WOODROW,2651.788889,6.855613e+05,295.153651,40.538840,-74.198260,700.000000,2.371646
FAR ROCKAWAY,2665.209091,5.269487e+05,241.696745,40.597624,-73.749532,413.000000,1.708753
RIVERDALE,20295.947368,2.818965e+06,444.068956,40.886288,-73.914183,745.833333,1.679544
PRINCE'S BAY,2470.840000,7.019019e+05,317.891725,40.526623,-74.197547,517.666667,1.628437
ARVERNE,2035.346154,4.067329e+05,203.628364,40.591684,-73.794277,260.780488,1.280669
CYPRESS HILLS,2339.777293,5.849109e+05,277.646468,40.681232,-73.886373,354.812500,1.277929
WILLIAMSBRIDGE,2716.093960,5.503399e+05,222.093402,40.878002,-73.862740,230.555556,1.038102


In [53]:
# Plot top 25 options 
top25_plot = top_25_neighborhoods.hvplot.scatter(
    x="PRICE PER SQFT",
    xlabel="Average sale price per square foot",
    y='AirBnb price per night',
    ylabel="Airbnb average price per night",
    hover_cols='all',
    by='neighbourhood',
    height=650,
    width=800,
    xformatter='%.0f',
    title = 'Sale price per SQFT vs. Airbnb Revenue per Night (Avg) by Neighborhood'
).opts(
legend_cols=6,
legend_position='bottom',
fontsize={'legend': 6,'title': 12}
)
top25_plot

:NdOverlay   [neighbourhood]
   :Scatter   [PRICE PER SQFT]   (AirBnb price per night,neighbourhood,GROSS SQUARE FEET,SALE PRICE,latitude,longitude,Average sale vs. Airbnb price Ratio)

In [54]:
#View top 25 location list to create new list based on Average Rent / Average Price Per SQFT
top25_neighborhood_list = top_25_neighborhoods.index
top25_neighborhood_list

Index(['WESTCHESTER SQUARE', 'SEA GATE', 'WOODROW', 'FAR ROCKAWAY',
       'RIVERDALE', 'PRINCE'S BAY', 'ARVERNE', 'CYPRESS HILLS',
       'WILLIAMSBRIDGE', 'PORT RICHMOND', 'STAPLETON', 'WAKEFIELD',
       'ELTINGVILLE', 'CITY ISLAND', 'HIGHBRIDGE', 'ST. ALBANS',
       'NEW BRIGHTON', 'PELHAM GARDENS', 'PARKCHESTER', 'FLATLANDS',
       'MORRISANIA', 'TODT HILL', 'KINGSBRIDGE', 'ROCKAWAY BEACH'],
      dtype='object', name='neighbourhood')

In [55]:
# List of neighborhood values to select the top 25 rows for each neighborhood
selected_neighborhoods = ['WESTCHESTER SQUARE', 'SEA GATE', 'WOODROW', 'FAR ROCKAWAY',
       'RIVERDALE', "PRINCE'S BAY", 'ARVERNE', 'CYPRESS HILLS',
       'WILLIAMSBRIDGE', 'PORT RICHMOND', 'STAPLETON', 'WAKEFIELD',
       'ELTINGVILLE', 'CITY ISLAND', 'HIGHBRIDGE', 'ST. ALBANS',
       'NEW BRIGHTON', 'PELHAM GARDENS', 'PARKCHESTER', 'FLATLANDS',
       'MORRISANIA', 'TODT HILL', 'KINGSBRIDGE', 'ROCKAWAY BEACH']

# Initialize an empty list to store the trimmed DataFrames
trimmed_dfs = []
trimmed_df = pd.concat([combined_data.loc[neighborhood] for neighborhood in selected_neighborhoods])
trimmed_df['SALE DATE'] = pd.to_datetime(trimmed_df['SALE DATE'])
trimmed_df['SALE YEAR'] = trimmed_df['SALE DATE'].dt.year
trimmed_df = trimmed_df.drop(columns={
    'SALE DATE',
    'GROSS SQUARE FEET',
    'SALE PRICE',
    'latitude',
    'longitude',
    'AirBnb price per night'
})
trimmed_df

,PRICE PER SQFT,SALE YEAR
neighbourhood,,
WESTCHESTER SQUARE,168.103448,2016
WESTCHESTER SQUARE,109.090909,2016
WESTCHESTER SQUARE,170.267197,2016
WESTCHESTER SQUARE,150.442478,2016
WESTCHESTER SQUARE,214.285714,2016
...,...,...
ROCKAWAY BEACH,235.697946,2017
ROCKAWAY BEACH,72.580645,2017
ROCKAWAY BEACH,327.198364,2017


In [56]:
# Switched orientation of dataframe to group by Sale Year with columns for each neighborhood using pivot table
trimmed_df_1 = trimmed_df.reset_index()
trimmed_df_1 = trimmed_df_1.groupby(['SALE YEAR', 'neighbourhood']).mean()
trimmed_df_1.head(5)
pivot_df = trimmed_df_1.reset_index().pivot_table(index='SALE YEAR', columns='neighbourhood', values='PRICE PER SQFT')
pivot_df

neighbourhood,ARVERNE,CITY ISLAND,CYPRESS HILLS,ELTINGVILLE,FAR ROCKAWAY,FLATLANDS,HIGHBRIDGE,KINGSBRIDGE,MORRISANIA,NEW BRIGHTON,...,RIVERDALE,ROCKAWAY BEACH,SEA GATE,ST. ALBANS,STAPLETON,TODT HILL,WAKEFIELD,WESTCHESTER SQUARE,WILLIAMSBRIDGE,WOODROW
SALE YEAR,,,,,,,,,,,,,,,,,,,,,
2016,197.774182,406.326167,279.969881,336.376726,241.512481,322.620039,165.517758,232.048946,184.407363,202.157170,...,461.74769,230.644515,288.532893,291.522390,215.952712,330.432769,241.272296,257.400752,202.786123,274.743203
2017,213.971762,324.925203,275.952177,353.309998,246.347682,294.653005,192.313541,251.057176,213.322234,228.960207,...,428.55586,222.032116,279.488409,275.854347,261.950464,385.968380,237.958902,223.639707,242.388627,303.208437


In [57]:
#Plot top_25 neighborhood average price per SQFT per year
top_bar_plot = pivot_df.hvplot.bar(
    x='SALE YEAR',
    y=['WESTCHESTER SQUARE', 'SEA GATE', 'WOODROW', 'FAR ROCKAWAY',
       'RIVERDALE', "PRINCE'S BAY", 'ARVERNE', 'CYPRESS HILLS',
       'WILLIAMSBRIDGE', 'PORT RICHMOND', 'STAPLETON', 'WAKEFIELD',
       'ELTINGVILLE', 'CITY ISLAND', 'HIGHBRIDGE', 'ST. ALBANS',
       'NEW BRIGHTON', 'PELHAM GARDENS', 'PARKCHESTER', 'FLATLANDS',
       'MORRISANIA', 'TODT HILL', 'KINGSBRIDGE', 'ROCKAWAY BEACH'],
    frame_width=650, 
    frame_height=450,
    rot=90,
    title='Housing Sale Price per Neighborhood (2016-2017)',
    fontscale = .8,
    ylim=[100,500]
)
top_bar_plot

:Bars   [SALE YEAR,neighbourhood]   (value)

In [58]:
grouped_by_neighborhood = trimmed_df.reset_index()
grouped_by_neighborhood = grouped_by_neighborhood.groupby(['neighbourhood', 'SALE YEAR']).mean()
#grouped_by_neighborhood.head(5)

grouped_by_neighborhood_2016 = trimmed_df[trimmed_df["SALE YEAR"]==2016]
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.reset_index()
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.groupby(['neighbourhood', 'SALE YEAR']).mean()
grouped_by_neighborhood_2017 = trimmed_df[trimmed_df["SALE YEAR"]==2017]
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.reset_index()
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.groupby(['neighbourhood', 'SALE YEAR']).mean()
display(grouped_by_neighborhood_2016.head(5))
display(grouped_by_neighborhood_2017.head(5))

,,PRICE PER SQFT
neighbourhood,SALE YEAR,
ARVERNE,2016,197.774182
CITY ISLAND,2016,406.326167
CYPRESS HILLS,2016,279.969881
ELTINGVILLE,2016,336.376726
FAR ROCKAWAY,2016,241.512481


,,PRICE PER SQFT
neighbourhood,SALE YEAR,
ARVERNE,2017,213.971762
CITY ISLAND,2017,324.925203
CYPRESS HILLS,2017,275.952177
ELTINGVILLE,2017,353.309998
FAR ROCKAWAY,2017,246.347682


In [59]:
import holoviews as hv

In [60]:
top_bar_plot_by_neighborhood_2016 = grouped_by_neighborhood_2016.hvplot.bar(
    x='neighbourhood',
    y ='PRICE PER SQFT',
    #by='SALE YEAR',
    frame_width=900, 
    frame_height=450,
    #rot=90,
    title='Housing Sale Price by Neighborhood over 2016-2017',
    fontscale=.8,
    color='#71797E',
    ylim=[100,500]
).opts(xrotation=90, 
fontsize={'title': 14}
)
top_bar_plot_by_neighborhood_2017 = grouped_by_neighborhood_2017.hvplot.bar(
    x='neighbourhood',
    y ='PRICE PER SQFT',
    #by='SALE YEAR',
    frame_width=900, 
    frame_height=450,
    #rot=90,
    title='Housing Sale Price by Neighborhood over 2016-2017',
    fontscale=.8,
    color='#3BB143',
    ylim=[100,500]
).opts(xrotation=90, 
fontsize={'title': 14}
)
top_bar_plot_by_neighborhood_2016 * top_bar_plot_by_neighborhood_2017  


:Overlay
   .Bars.I  :Bars   [neighbourhood]   (PRICE PER SQFT)
   .Bars.II :Bars   [neighbourhood]   (PRICE PER SQFT)

In [61]:
#Calculate % Change in Price between 2016 and 2017 in top 25 neighborhoods for purchase:
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.reset_index()
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.rename(columns={'PRICE PER SQFT':'PRICE PER SQFT 2016'})
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.set_index('neighbourhood')
grouped_by_neighborhood_2016 = grouped_by_neighborhood_2016.drop(columns=['SALE YEAR'])
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.reset_index()
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.rename(columns={'PRICE PER SQFT':'PRICE PER SQFT 2017'})
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.set_index('neighbourhood')
grouped_by_neighborhood_2017 = grouped_by_neighborhood_2017.drop(columns=['SALE YEAR'])



In [62]:
#Combine dataframes
sale_price_percent_change = pd.concat([grouped_by_neighborhood_2016,grouped_by_neighborhood_2017], axis='columns',join='inner')


,PRICE PER SQFT 2016,PRICE PER SQFT 2017
neighbourhood,,
ARVERNE,197.774182,213.971762
CITY ISLAND,406.326167,324.925203
CYPRESS HILLS,279.969881,275.952177
ELTINGVILLE,336.376726,353.309998
FAR ROCKAWAY,241.512481,246.347682
FLATLANDS,322.620039,294.653005
HIGHBRIDGE,165.517758,192.313541
KINGSBRIDGE,232.048946,251.057176
MORRISANIA,184.407363,213.322234


In [70]:
#Calculate percent change
sale_price_percent_change['YoY Sale Price % Change'] = round((sale_price_percent_change['PRICE PER SQFT 2017']-sale_price_percent_change['PRICE PER SQFT 2016']) /sale_price_percent_change['PRICE PER SQFT 2017']*100,2)
sale_price_percent_change = sale_price_percent_change.sort_values(by='YoY Sale Price % Change')
sale_price_percent_change

,PRICE PER SQFT 2016,PRICE PER SQFT 2017,YoY Sale Price % Change
neighbourhood,,,
CITY ISLAND,406.326167,324.925203,-25.05
WESTCHESTER SQUARE,257.400752,223.639707,-15.10
FLATLANDS,322.620039,294.653005,-9.49
RIVERDALE,461.747690,428.555860,-7.75
ST. ALBANS,291.522390,275.854347,-5.68
ROCKAWAY BEACH,230.644515,222.032116,-3.88
SEA GATE,288.532893,279.488409,-3.24
PORT RICHMOND,239.827872,233.182807,-2.85
PARKCHESTER,253.699790,247.056018,-2.69


### Make Fake Investment Portfolio

In [73]:
investment_selection= pd.concat([top_25_neighborhoods,sale_price_percent_change], axis='columns',join='inner')
investment_selection

,GROSS SQUARE FEET,SALE PRICE,PRICE PER SQFT,latitude,longitude,AirBnb price per night,Average sale vs. Airbnb price Ratio,PRICE PER SQFT 2016,PRICE PER SQFT 2017,YoY Sale Price % Change
neighbourhood,,,,,,,,,,
WESTCHESTER SQUARE,3146.084337,7.039822e+05,232.697507,40.843780,-73.844690,670.000000,2.879275,257.400752,223.639707,-15.10
SEA GATE,2722.170732,7.551646e+05,285.298160,40.577053,-74.008880,787.000000,2.758518,288.532893,279.488409,-3.24
WOODROW,2651.788889,6.855613e+05,295.153651,40.538840,-74.198260,700.000000,2.371646,274.743203,303.208437,9.39
FAR ROCKAWAY,2665.209091,5.269487e+05,241.696745,40.597624,-73.749532,413.000000,1.708753,241.512481,246.347682,1.96
RIVERDALE,20295.947368,2.818965e+06,444.068956,40.886288,-73.914183,745.833333,1.679544,461.747690,428.555860,-7.75
PRINCE'S BAY,2470.840000,7.019019e+05,317.891725,40.526623,-74.197547,517.666667,1.628437,299.303611,344.447151,13.11
ARVERNE,2035.346154,4.067329e+05,203.628364,40.591684,-73.794277,260.780488,1.280669,197.774182,213.971762,7.57
CYPRESS HILLS,2339.777293,5.849109e+05,277.646468,40.681232,-73.886373,354.812500,1.277929,279.969881,275.952177,-1.46
WILLIAMSBRIDGE,2716.093960,5.503399e+05,222.093402,40.878002,-73.862740,230.555556,1.038102,202.786123,242.388627,16.34


In [90]:
final_plot = investment_selection.hvplot.points(
    "longitude", 
    "latitude", 
    geo=True,
    size='Average sale vs. Airbnb price Ratio',
    color='YoY Sale Price % Change',
    tiles="OSM", 
    scale=20,
    frame_width=650, 
    frame_height=450,
    title='Sale Price to Airbnb Price Ratio vs % Change in Sale Prices',
    xlabel='Longitude',
    ylabel='Latitude'
)
final_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (YoY Sale Price % Change,Average sale vs. Airbnb price Ratio)

In [87]:
final_plot = investment_selection.hvplot.points(
    "longitude", 
    "latitude", 
    geo=True,
    size='Average sale vs. Airbnb price Ratio',
    color='YoY Sale Price % Change',
    tiles="ESRI", 
    scale=20,
    frame_width=650, 
    frame_height=450,
    title='Sale Price to Airbnb Price Ratio vs % Change in Sale Prices',
)
final_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (YoY Sale Price % Change,Average sale vs. Airbnb price Ratio)

In [91]:
url = 'https://www.zillow.com/westchester-square-bronx-new-york-ny/?searchQueryState=%7B%22mapBounds%22%3A%7B%22north%22%3A40.85310659184466%2C%22east%22%3A-73.82713394430543%2C%22south%22%3A40.828074666924685%2C%22west%22%3A-73.86408405569459%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A15%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A403111%2C%22regionType%22%3A31%7D%5D%2C%22pagination%22%3A%7B%7D%7D'
